In [7]:
from fastapi import APIRouter, HTTPException
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re
import json
import ast
import requests

In [16]:
print(str(user_project_info[13]))

{'projectInfoId': 30, 'workspaceId': 39, 'title': '리듬게임 채보 분석기', 'category': '웹앱', 'targetUsers': ['리듬게임 초보 분석가', '게임 개발자', '음악 교육자'], 'coreFeatures': ['영상 기반 채보 분석 및 DB 저장', '분야별 점수 계산 및 분석', '사용자 맞춤형 곡 추천', '분석 결과 시각화 및 CSV 다운로드'], 'technologyStack': ['Python (OpenCV, Scikit-learn)', 'Spring Boot', 'React', 'PostgreSQL', 'Docker'], 'problemSolving': {'currentProblem': '리듬게임 분석가들이 채보를 수작업으로 분석하는 데 많은 시간과 노력이 소모되며, 정확한 피드백을 제공하기 어렵다.', 'solutionIdea': '본 프로젝트는 컴퓨터 비전을 활용하여 리듬게임 영상을 프레임 단위로 분석하고, 노트의 타이밍과 종류를 자동으로 분류하여 데이터베이스에 저장합니다. 이후, Scikit-learn을 통해 각 분야에 대한 점수를 계산하고, 사용자가 보완이 필요한 영역을 파악하여 적합한 연습 곡을 추천합니다. 이 모든 과정은 Spring Boot와 React를 통해 사용자에게 시각화된 형태로 제공되며, 사용자 경험을 극대화하기 위해 API 응답 속도와 분석 정확도를 철저히 관리합니다.', 'expectedBenefits': ['사용자는 자동화된 분석을 통해 시간과 노력을 절약할 수 있다.', '정확한 데이터 기반 피드백으로 개인의 연습 효과를 극대화할 수 있다.', '리듬게임 관련 교육 및 연구에 기여하여 시장 내 경쟁력을 높일 수 있다.']}}


In [3]:
# 기존 requests 변수 삭제
import requests
import ast
import pandas as pd

DB_url = 'http://13.125.204.95:8080/api/workspaces/project-info'
response = requests.get(url = DB_url)
data = response.json()['data']
user_project_info = ast.literal_eval(data)

workspaceID_lst = []
solutionIdea_lst = []
stack_lst = []

for i in range(len(user_project_info)) :
    workspaceID_lst.append(user_project_info[i]['workspaceId'])
    solutionIdea_lst.append(user_project_info[i]['problemSolving']['solutionIdea'])
    stack_lst.append(user_project_info[i]['technologyStack'])

send_dataset = pd.DataFrame(index = workspaceID_lst, data = {
    'subject': solutionIdea_lst,
    'stack': stack_lst
})

In [ ]:

# 텍스트 전처리 함수
def preprocess_text(text):
    if pd.isna(text):
        return ""
    # HTML 태그 제거
    text = re.sub('<.*?>', '', str(text))
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)
    text = re.sub('[^가-힣a-zA-Z0-9\s]', ' ', text)
    # 연속된 공백을 하나로 변환
    text = re.sub('\s+', ' ', text)
    return text.strip()

# TfidfVectorizer 진행
vectorizer = TfidfVectorizer(
    max_features=1000,  # 최대 특성 수
    ngram_range=(1, 2),  # 1-gram과 2-gram 사용
    min_df=1,  # 최소 문서 빈도
    max_df=0.95,  # 최대 문서 빈도 (너무 자주 나오는 단어 제외)
    stop_words=None
)


# 2. 데이터 중에 스택이 포함되있는 내용만 추출
def filter_stack_recomend_subjects(input_text ,target_stack, top_k = 10) :
    # 공통되는 요소가 하나라도 있는 행
    has_stack = sample_df[sample_df['stack'].apply(lambda x: bool(set(x) & set(target_stack)))]

    # 원본 인덱스 저장
    original_indices = has_stack.index.tolist()

    # 벡터화 실행
    tfidf_matrix = vectorizer.fit_transform(has_stack['processed_subject'])

    # 사용자의 특정 개요의 유사도 기반 추천
    processed_input = preprocess_text(input_text)
    input_vector = vectorizer.transform([processed_input])
    similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
    similar_indices = similarities.argsort()[::-1][:top_k]

    # 원본 DataFrame의 인덱스로 변환
    original_similar_indices = [original_indices[i] for i in similar_indices]

    return original_similar_indices

In [15]:
has_stack

,subject,stack
47,이 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹의 효율적인 운영을 지원합...,"[HTML, CSS, JavaScript, Java (Servlet, JSP), M..."
17,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공,"[javaScript, java]"
18,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있도록 지원,"[javaScript, java]"
23,스터디 그룹 내에서 과제 관리와 퀴즈 기능을 통해 학습 효과를 극대화,"[javaScript, java]"
25,스터디 그룹 내 학습 내용을 복습할 수 있는 기능 제공 및 소통 강화,"[스프링, 리액트, MySql]"
40,"이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고, 스터디 그룹을 통해 협력할 수...","[React, Spring Boot, MariaDB, Redis, Docker, K..."
34,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 스터디 그룹 내에서의 소통을 강화...,"[HTML, CSS, JavaScript, Java (Servlet, JSP), M..."
26,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,"[React, TypeScript, Spring Boot, PostgreSQL, A..."
46,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 학생들이 쉽게 스터디 그룹을 생성...,"[HTML, CSS, JavaScript, Java (Spring Framework..."
38,"본 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹 운영을 지원하며, 체크리...","[HTML, CSS, JavaScript, Java (Spring Framework..."


In [27]:
sample_df

,subject,stack,processed_subject,sim_score
47,이 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹의 효율적인 운영을 지원합...,NaN,이 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹의 효율적인 운영을 지원합...,0.182253
17,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공,NaN,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공,0.032866
18,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있도록 지원,NaN,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있도록 지원,0.013889
23,스터디 그룹 내에서 과제 관리와 퀴즈 기능을 통해 학습 효과를 극대화,NaN,스터디 그룹 내에서 과제 관리와 퀴즈 기능을 통해 학습 효과를 극대화,0.017371
25,스터디 그룹 내 학습 내용을 복습할 수 있는 기능 제공 및 소통 강화,NaN,스터디 그룹 내 학습 내용을 복습할 수 있는 기능 제공 및 소통 강화,0.022420
40,"이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고, 스터디 그룹을 통해 협력할 수...",NaN,이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고 스터디 그룹을 통해 협력할 수 ...,0.179384
34,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 스터디 그룹 내에서의 소통을 강화...,NaN,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 스터디 그룹 내에서의 소통을 강화...,0.067211
26,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,NaN,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,0.162597
46,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 학생들이 쉽게 스터디 그룹을 생성...,NaN,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 학생들이 쉽게 스터디 그룹을 생성...,0.181220
38,"본 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹 운영을 지원하며, 체크리...",NaN,본 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹 운영을 지원하며 체크리스...,0.093784


In [30]:
send_dataset

,subject,stack
47,이 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹의 효율적인 운영을 지원합...,"[HTML, CSS, JavaScript, Java (Servlet, JSP), M..."
17,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있는 기능 제공,"[javaScript, java]"
18,스터디 그룹 내에서 과제 관리와 퀴즈를 통해 학습 내용을 반복 학습할 수 있도록 지원,"[javaScript, java]"
23,스터디 그룹 내에서 과제 관리와 퀴즈 기능을 통해 학습 효과를 극대화,"[javaScript, java]"
25,스터디 그룹 내 학습 내용을 복습할 수 있는 기능 제공 및 소통 강화,"[스프링, 리액트, MySql]"
40,"이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고, 스터디 그룹을 통해 협력할 수...","[React, Spring Boot, MariaDB, Redis, Docker, K..."
34,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 스터디 그룹 내에서의 소통을 강화...,"[HTML, CSS, JavaScript, Java (Servlet, JSP), M..."
26,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,"[React, TypeScript, Spring Boot, PostgreSQL, A..."
46,이 플랫폼은 사용자 친화적인 인터페이스를 제공하여 학생들이 쉽게 스터디 그룹을 생성...,"[HTML, CSS, JavaScript, Java (Spring Framework..."
38,"본 플랫폼은 사용자 친화적인 인터페이스를 통해 스터디 그룹 운영을 지원하며, 체크리...","[HTML, CSS, JavaScript, Java (Spring Framework..."


In [49]:
target_stack = ['React', 'vue']
recommendation_threshold = 0.1

input_text = "이 프로젝트는 메이플스토리의 공식 API를 통해 실시간으로 경매장 데이터를 수집하고, ARIMA 모델을 활용하여 과거 데이터를 기반으로 시세를 예측합니다. 사용자들은 관심 있는 아이템의 시세 변동을 시각적으로 확인하고, 가격 변동에 대한 알림을 설정할 수 있습니다. 또한, 데이터베이스는 매주 자동 백업되며, 시스템 모니터링 대시보드를 통해 실시간 성능을 확인할 수 있습니다. 이러한 기능들은 사용자 경험을 극대화하고, 예측 정확도를 높여 거래의 효율성을 증가시킵니다."

# 공통되는 요소가 하나라도 있는 행
sample_df = send_dataset.copy()
target_stack = [item.lower() for item in target_stack]
sample_df['stack'] = sample_df['stack'].astype(str).str.lower().apply(ast.literal_eval)
sample_df['processed_subject'] = sample_df['subject'].apply(preprocess_text)

# 벡터화 실행
tfidf_matrix = vectorizer.fit_transform(sample_df['processed_subject'])

# 사용자의 특정 개요의 유사도 검사
processed_input = preprocess_text(input_text)
input_vector = vectorizer.transform([processed_input])
similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
sample_df['sim_score'] = similarities

# 같은 스택 있는지에 대한 필터
has_stack = sample_df[sample_df['stack'].apply(lambda x: bool(set(x) & set(target_stack)))]
has_stack_sort = has_stack.sort_values(by = 'sim_score', ascending= False)
has_stack_filter = has_stack_sort[has_stack_sort['sim_score'] > recommendation_threshold].head(10)
has_stack_filter.index.tolist()

[40, 26, 62, 45]

In [41]:
"RECOMMENDATION_THRESHOLD".lower()

'recommendation_threshold'

In [22]:
has_stack

,subject,stack,processed_subject,sim_score


In [57]:
from fastapi import APIRouter, HTTPException
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re
import json
import ast
import requests

project_info = "{'projectInfoId': 30, 'workspaceId': 39, 'title': '리듬게임 채보 분석기', 'category': '웹앱', 'targetUsers': ['리듬게임 초보 분석가', '게임 개발자', '음악 교육자'], 'coreFeatures': ['영상 기반 채보 분석 및 DB 저장', '분야별 점수 계산 및 분석', '사용자 맞춤형 곡 추천', '분석 결과 시각화 및 CSV 다운로드'], 'technologyStack': ['Python (OpenCV, Scikit-learn)', 'Spring Boot', 'React', 'PostgreSQL', 'Docker'], 'problemSolving': {'currentProblem': '리듬게임 분석가들이 채보를 수작업으로 분석하는 데 많은 시간과 노력이 소모되며, 정확한 피드백을 제공하기 어렵다.', 'solutionIdea': '본 프로젝트는 컴퓨터 비전을 활용하여 리듬게임 영상을 프레임 단위로 분석하고, 노트의 타이밍과 종류를 자동으로 분류하여 데이터베이스에 저장합니다. 이후, Scikit-learn을 통해 각 분야에 대한 점수를 계산하고, 사용자가 보완이 필요한 영역을 파악하여 적합한 연습 곡을 추천합니다. 이 모든 과정은 Spring Boot와 React를 통해 사용자에게 시각화된 형태로 제공되며, 사용자 경험을 극대화하기 위해 API 응답 속도와 분석 정확도를 철저히 관리합니다.', 'expectedBenefits': ['사용자는 자동화된 분석을 통해 시간과 노력을 절약할 수 있다.', '정확한 데이터 기반 피드백으로 개인의 연습 효과를 극대화할 수 있다.', '리듬게임 관련 교육 및 연구에 기여하여 시장 내 경쟁력을 높일 수 있다.']}}"
top_k = 10


##################################################
# DB에 이으는 과정 필요! (DB 연결 구역)
##################################################

# 임시 데이터로 작성
# project_info DB 연결필요

def read_DB() :
    # url 접속
    DB_url = 'http://13.125.204.95:8080/api/workspaces/project-info'
    response = requests.get(url = DB_url)
    data = response.json()['data']

    # 방법 1: JSON string을 .json 확장자로 저장 (기본)
    user_project_info = ast.literal_eval(data)

    workspaceID_lst = []
    solutionIdea_lst = []
    stack_lst = []

    for i in range(len(user_project_info)) :
        workspaceID_lst.append(user_project_info[i]['workspaceId'])
        solutionIdea_lst.append(user_project_info[i]['problemSolving']['solutionIdea'])
        stack_lst.append(user_project_info[i]['technologyStack'])

    send_dataset = pd.DataFrame(index = workspaceID_lst, data = {
        'subject': solutionIdea_lst,
        'stack': stack_lst
    })

    return send_dataset
####################################################

# 텍스트 전처리 함수
def preprocess_text(text):
    if pd.isna(text):
        return ""
    # HTML 태그 제거
    text = re.sub('<.*?>', '', str(text))
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)
    text = re.sub('[^가-힣a-zA-Z0-9\s]', ' ', text)
    # 연속된 공백을 하나로 변환
    text = re.sub('\s+', ' ', text)
    return text.strip()

# TfidfVectorizer 진행
vectorizer = TfidfVectorizer(
    max_features=1000,  # 최대 특성 수
    ngram_range=(1, 2),  # 1-gram과 2-gram 사용
    min_df=1,  # 최소 문서 빈도
    max_df=0.95,  # 최대 문서 빈도 (너무 자주 나오는 단어 제외)
    stop_words=None
)

class ProjectRecommender :
    def __init__(self,  sample_df, vectorizer):
        self.sample_df = sample_df
        self.vectorizer = vectorizer

        # 전처리를 __init__에서 미리 수행
        self.sample_df['processed_subject'] = self.sample_df['subject'].apply(preprocess_text)

    def filter_stack_recomend_subjects(self, input_text, target_stack, score, top_k = 10) :
        
        df_copy = self.sample_df.copy()
        
        # stack 관련 단어 데이터 소문자화 
        target_stack = [item.lower() for item in target_stack]
        df_copy['stack'] = df_copy['stack'].astype(str).str.lower().apply(ast.literal_eval)

        # 벡터화 실행
        tfidf_matrix = vectorizer.fit_transform(df_copy['processed_subject'])

        # 사용자의 특정 개요의 유사도 기반 추천
        processed_input = preprocess_text(input_text)
        input_vector = vectorizer.transform([processed_input])
        similarities = cosine_similarity(input_vector, tfidf_matrix).flatten()
        df_copy['sim_score'] = similarities


        # 같은 스택 있는지에 대한 필터
        has_stack = df_copy[df_copy['stack'].apply(lambda x: bool(set(x) & set(target_stack)))]
        has_stack_sort = has_stack.sort_values(by = 'sim_score', ascending= False)
        
        ## 이게 문제
        has_stack_filter = has_stack_sort[has_stack_sort['sim_score'] > score]
        # has_stack_filter = has_stack_sort.copy()
        original_similar_indices = has_stack_filter.index.tolist()

        return original_similar_indices

# 데이터 준비
send_dataset = read_DB()

sample_df = send_dataset.copy()

# 추천 시스템 초기화
recommender = ProjectRecommender(sample_df, vectorizer)

# 요청에서 데이터 추출
input_text = ast.literal_eval(project_info)['problemSolving']['solutionIdea']
target_stack = ast.literal_eval(project_info)['technologyStack']
top_k = top_k
score = 0.1

recommend_id = recommender.filter_stack_recomend_subjects(input_text, target_stack, score, top_k)

similer_ID = {"project_ID" : recommend_id}
similer_ID

{'project_ID': [39, 45]}

In [46]:
has_stack

,subject,stack,processed_subject,sim_score
40,"이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고, 스터디 그룹을 통해 협력할 수...","[react, spring boot, mariadb, redis, docker, k...",이 플랫폼은 취업 준비생들이 서로의 경험을 공유하고 스터디 그룹을 통해 협력할 수 ...,0.033372
26,이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,"[react, typescript, spring boot, postgresql, a...",이 웹 기반 프로젝트 관리 플랫폼은 초보 개발자들이 쉽게 사용할 수 있도록 직관적인...,0.018435
62,FocusForge는 사용자의 행동 로그를 실시간으로 분석하여 집중 시간과 산만 행...,"[react, fastapi, postgresql, langchain + opena...",FocusForge는 사용자의 행동 로그를 실시간으로 분석하여 집중 시간과 산만 행...,0.020791
69,프프는 사용자 데이터를 분석하여 개인 맞춤형 추천을 제공하는 플랫폼입니다. 사용자는...,"[react, node.js, mongodb, aws, graphql]",프프는 사용자 데이터를 분석하여 개인 맞춤형 추천을 제공하는 플랫폼입니다 사용자는 ...,0.019916
39,"본 프로젝트는 컴퓨터 비전을 활용하여 리듬게임 영상을 프레임 단위로 분석하고, 노트...","[python (opencv, scikit-learn), spring boot, r...",본 프로젝트는 컴퓨터 비전을 활용하여 리듬게임 영상을 프레임 단위로 분석하고 노트의...,1.000000
45,"이 프로젝트는 컴퓨터 비전 기술을 활용하여 리듬게임 영상에서 채보를 인식하고, 이를...","[react, spring, python, opencv, postgresql, aw...",이 프로젝트는 컴퓨터 비전 기술을 활용하여 리듬게임 영상에서 채보를 인식하고 이를 ...,0.127914
49,"이 앱은 스마트 기기와 연동하여 반려동물의 활동량, 수면 시간, 식사 및 배변 패턴...","[flutter, firebase, tensorflow lite, aws iot, ...",이 앱은 스마트 기기와 연동하여 반려동물의 활동량 수면 시간 식사 및 배변 패턴을 ...,0.036595
51,"AI 알고리즘을 통해 사용자의 음식 선호도, 예산, 위치, 분위기를 종합적으로 분석...","[react native, django (python), postgresql, aw...",AI 알고리즘을 통해 사용자의 음식 선호도 예산 위치 분위기를 종합적으로 분석하여 ...,0.028221
